In [1]:
import can

channel_id = 0
bitrate = 500000

# 使用正确的参数调用Bus构造函数
bus = can.Bus(channel=channel_id, bustype='kvaser', bitrate=bitrate)


In [2]:
# 接收消息
message = bus.recv(1) 

if message:
    print(f"接收到消息: {message}")
else:
    print("等待消息超时。")


接收到消息: Timestamp: 1711167796.463741    ID:      000    S Rx                DL:  8    66 ef 17 27 b5 ef 9d 93     Channel: 0


In [89]:
# 创建一个CAN消息
msg = can.Message(arbitration_id=0x18FF7000, data=[0x1, 0x2, 0x3], is_extended_id=False)

# 发送消息
bus.send(msg)


In [104]:
import can

def create_can_message(arbitration_id, set_para_num, heart, para_value):
    """
    CAN 2.0B, Extended Data Identifier
    ID: 0x18FF7000
    :param arbitration_id: 整数，消息的仲裁ID。

    :param set_para_num: 整数，设置参数的编号，取值应为0-30。
    :param heart: (bool): 如果为True，则心跳有效；否则无效。
    :para para_value: 浮点数，取值应为0~9999.99, 单位mm
    :return: `can.Message`对象。
    """
    if not 0 <= set_para_num <= 30:
        raise ValueError("set_para_num must be in range 0-30.")
    if not 0 <= para_value <= 9999.99:
        raise ValueError("para_value must be in range 0 to 9999.99.")
    
    set_para_num_byte = set_para_num.to_bytes(1, 'little')
    para_value_int = int(para_value * 100)  

    # 创建8字节的数据
    data_bytes = bytearray(8)
    data_bytes[0] = set_para_num_byte[0] & 0xFF  
    
    # 位操作
    mask_to_clear = 0xFC
    data_bytes[1] &= mask_to_clear
    heartbeat_code = 0x01 if heart else 0x00  # 0x01 for True, 0x00 for False
    data_bytes[1] = heartbeat_code & 0xFF
    
    data_bytes[3] = set_para_num_byte[0] & 0xFF  

    # 取小端序
    data_bytes[7] = (para_value_int & 0xFF)  # 取value的最低8位
    data_bytes[6] = (para_value_int >> 8) & 0xFF  # 取value的次低8位
    data_bytes[5] = (para_value_int >> 16) & 0xFF  # 取value的次高8位
    data_bytes[4] = (para_value_int >> 24) & 0xFF  # 取value的最高8位
    
    # 构建消息
    return can.Message(arbitration_id=arbitration_id, data=data_bytes, is_extended_id=True)


message = create_can_message(0x18FF7000, 17, True, 1234.56)
bus.send(message)